# Τεχνικές Εξόρυξης Δεδομένων
# Μέλη: 
## 1. Αποστολοπούλου Αλεξάνδρα (1115201700005)
## 2. Μπριάκος Σπυρίδων (1115201700101)
## Εργασία 3η

In [7]:
import os
import glob
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer 
from wordcloud import WordCloud, STOPWORDS
from sklearn.metrics.pairwise import cosine_similarity
import warnings
import time
import sys
import string
import math
import string
import nltk
from nltk import BigramCollocationFinder
from nltk import BigramAssocMeasures
from nltk.collocations import *

## Ερώτημα 3.1

In [8]:
"""
Σε αυτό το ερώτημα φτιάξαμε τον πίνακα  TF-IDF
των unigrams και των bigrams για τη νέα στήλη,
αφαιρώντας τα stop words.
"""

t1 = time.time()
warnings.simplefilter("ignore", UserWarning)

path = '/home/alexa/Documents/Data_Mining/ERGASIA_1/train.csv'
path1 = '/home/alexa/Desktop/greek_stopwords.txt'

df = pd.read_csv(path)
df_new = df[['id','name','description']].copy()
df_new['new_col'] = df_new['name'] + df_new['description'] 
df_new.dropna(axis=0,inplace=True)

#Get rid of the punctiations and the symbols.
for i, row in df_new.iterrows():
    df_new.loc[i,'new_col']  =  df_new.loc[i,'new_col'].lower()
    df_new.loc[i,'new_col'] = ''.join(ch for ch in df_new['new_col'][i] if ch not in set(string.punctuation))
    
greek_stopwords = (pd.read_csv(path1,sep='\n'))
greek_stopwords.columns = ['words']
greek_stopwords = greek_stopwords['words'].tolist()

all_stopwords = list(STOPWORDS) + greek_stopwords

vectorizer = TfidfVectorizer(use_idf=True,stop_words=all_stopwords,ngram_range=(1,2),max_features=1000)
V = vectorizer.fit_transform(df_new['new_col'].tolist())

# print(vectorizer.vocabulary_)
# print(V.toarray())

## Ερώτημα 3.2

In [9]:
"""
Εδώ βρίσκουμε το Cosine Similarity 
όλων των ακινήτων μεταξύ τους, καθώς
και τα 100 πιο όμοια αποθηκευμένα σε
ένα dictionary. 
"""


V = V.toarray()
sim = cosine_similarity(V)
np.fill_diagonal(sim,0)
print('Cosine Similarity Matrixs shape: ', sim.shape)

#Select the upper triangle of the array because (x,y)=(y,x) refers to the same estates.
sim = np.triu(sim)

print('Number of 1 into Cosine Similarity Matrix: ', np.count_nonzero(sim==1))
#Convert sim matrix to array and find indices of 100 greatest values of our array.
indices = np.argpartition(sim.flatten(), -100)[-100:]
#Find the original indices of sim matrix of 100 greatest values.
original_indexes = np.unravel_index(indices,sim.shape)
ziped_indexes = list(zip(original_indexes[0],original_indexes[1]))

#Creating dictionary of 100 pairs of most similar pairs. (key: (id1,id2), value: score)
final_dict = {}
for x,y in ziped_indexes:
    score = sim[x][y]
    pair = (df_new['id'][x],df_new['id'][y])
    final_dict[pair] = math.floor(score) 
# print(final_dict)

Cosine Similarity Matrixs shape:  (9552, 9552)
Number of 1 into Cosine Similarity Matrix:  181


## Ερώτημα 3.3

In [10]:
"""
Σε αυτό το ερώτημα φτιάξαμε μία συνάρτηση 
η οποία παίρνει σαν είσοδο ένα id και έναν
ακέραιο αριθμό N, και επιστρέφει τα Ν πιο
όμοια ακίνητα.
"""

def recommend(item_id, N):
    id_list = df_new['id'].tolist()
    index = id_list.index(item_id)
    sim_list = sim[index]
    indices = (sorted(range(len(sim_list)), key = lambda sub: sim_list[sub])[-1:-N-1:-1])
    print('Based to: ', df_new['name'][index], ' \nDescription: ', df_new['description'][index], '\n\n')
    print('Recommending ', N, ' apartments similar to ', item_id, '\n--------------------------------------------')
    for y in indices:
        print('Recommended: ', df_new['name'][y], '\nDescription: ', df_new['description'][y], '\n')
recommend(10595,5)

Based to:  96m2, 3BR, 2BA, Metro, WI-FI etc...  
Description:  Athens Furnished Apartment No6 is 3-bedroom apartment with 2-bathrooms -excellent located  -close to metro station,  -lovely,  -very clean  with all the facilities that you will need, nice balcony, excellent Wi-Fi, cable tv, fully air conditioned… Athens Furnished Apartment No6 is an excellent located, close to metro, lovely, very clean 3-bedroom apartment with 2-bathrooms with all the facilities that you will need and balcony. It is on the 2nd floor but do not worry because there is elevator in the building. Fully equipped kitchen with everything you need to prepare your lunch/dinner. Living room to relax and enjoy a movie or a sport event. 2 Clean nice bathrooms. For more than 6 people there is a sofa/bed.  Apartment No6 has everything you will need. 1st Bedroom – Double bed 2nd Bedroom – 2 single beds 3rd Bedroom – 2 single beds -Telephone line for incoming calls or to call us if you need something. -Free fast Wi-Fi from

## Ερώτημα 3.4

In [11]:
"""
Εδώ χρησιμοποιήσαμε τον BigramCollocationFinder 
για να βρούμε τα 10 words που τείνουν να 
εμφανίζονται συχνά μαζί.
"""

filter_stops = lambda w: len(w) < 3 or w in all_stopwords 
text = nltk.wordpunct_tokenize(' '.join([str(elem) for elem in df_new['new_col'].tolist()]))
text = (ch for ch in text if ch not in set(string.punctuation))
text = [w.lower() for w in text]
    
biagram_collocation = BigramCollocationFinder.from_words(text)
biagram_collocation.apply_word_filter(filter_stops) 
biagram_collocation.nbest(BigramAssocMeasures.likelihood_ratio, 10) 

[('living', 'room'),
 ('fully', 'equipped'),
 ('metro', 'station'),
 ('walking', 'distance'),
 ('double', 'bed'),
 ('washing', 'machine'),
 ('equipped', 'kitchen'),
 ('syntagma', 'square'),
 ('brand', 'new'),
 ('air', 'conditioning')]

In [12]:
t2 = time.time()
print("Time elapsed: ",t2-t1)

Time elapsed:  34.558918952941895
